In [1]:
import pandas as pd             # data package
import matplotlib.pyplot as plt # graphics 
import datetime as dt
import numpy as np
from census import Census # This is new...

import requests, io             # internet and input tools  
import zipfile as zf            # zip file tools 
import os  

#import weightedcalcs as wc
#import numpy as np

import pyarrow as pa
import pyarrow.parquet as pq
 
from bokeh.palettes import brewer, Spectral6
from bokeh.io import show, output_file, curdoc
from bokeh.plotting import figure
from bokeh.models import ColumnDataSource, HoverTool, Panel, Tabs, GeoJSONDataSource, LinearColorMapper
from bokeh.models import ColorBar
from bokeh.layouts import column, gridplot, row
from bokeh.transform import factor_cmap
from bokeh.models import NumeralTickFormatter, Title, Label, Paragraph, Div, CustomJSHover, BoxAnnotation

In [29]:
tariff_list = pd.read_csv("./tariff-lists/canada-march-list.csv", header = None)

tariff_list.columns = ['hs-code']


In [9]:
tariff_list.head()

,hs-code
0,0105.11.22
1,0105.94.92
2,0105.99.12
3,0207.11.91
4,0207.11.92


In [ ]:
def fix_list(tariff_list):
    
    foo = tariff_list.copy(deep=True)
    
    foo["hs-code"] = foo["hs-code"].astype(str)
    
    # Remove periods from hs-code
    foo["hs-code"] = foo["hs-code"].str.replace(".", "")
    
    for index, row in foo.iterrows():
        
        if len(row["hs-code"]) < 8:
            foo.loc[index, "hs-code"] = "0" + row["hs-code"]
            
    foo["hs6"] = foo["hs-code"].str[0:6]

    print(foo["hs6"].iloc[1:5])

    out = foo.groupby(["hs6"]).agg({"hs6":"first"})

    return out
            

In [46]:
canada_list = fix_list(tariff_list)



1    01059492
2    01059912
3    02071191
4    02071192
Name: hs8, dtype: object


In [50]:
canada_list.head(25)

,hs8
hs8,
01051122,01051122
01059492,01059492
01059912,01059912
02071191,02071191
02071192,02071192
02071291,02071291
02071292,02071292
02071392,02071392
02071393,02071393


In [57]:
def get_aggregate_hs6_exports(ecom):
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,CTY_CODE,COMM_LVL,E_COMMODITY_SDESC"
    
    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS6" + "&E_COMMODITY=" + ecom + "*"
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + ""

    r = requests.get(url) 

    #print(r.status_code)

    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m") 

    df["exports"] = df["ALL_VAL_MO"].astype(float)

    return df[df["CTY_NAME"] == "TOTAL FOR ALL COUNTRIES"].exports.sum()

In [58]:
get_aggregate_hs6_exports("020727")

134192759.0

In [ ]:
def get_aggregate_exports():
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,E_COMMODITY,E_COMMODITY_SDESC"

    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS2" 
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + "1220"

    r = requests.get(url) 

    print(r.status_code)

    df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

    df.columns = r.json()[0]

    df.time = pd.to_datetime(df.time, format="%Y-%m") 

    df["exports"] = df["ALL_VAL_MO"].astype(float)

    return df.exports.sum()

In [19]:
def get_exports_hs10(ecom, tariff):
    
    my_key = "&key=34e40301bda77077e24c859c6c6c0b721ad73fc7"

    end_use = "hs?get=CTY_NAME,ALL_VAL_MO,CTY_CODE,COMM_LVL,E_COMMODITY_SDESC"
    
    surl = "https://api.census.gov/data/timeseries/intltrade/exports/" + end_use 

    surl  = surl + my_key + "&time=" + "from+2024-01+to+2024-12" + "&COMM_LVL=HS6" + "&E_COMMODITY=" + ecom + "*"
    # the issue is that uniform HScodes are at the HS6 level, after that they may differ by country
    # the China ones are at HS8 and appear different, so we need to go back to the HS6 level

    url = surl + "&CTY_CODE=" + "1220"

    # Canada is 1220

    r = requests.get(url) 
    
    #print(r.status_code)
    
    if r.status_code == 200:
    
        df = pd.DataFrame(r.json()[1:]) # This then converts it to a dataframe

        df.columns = r.json()[0]

        df.time = pd.to_datetime(df.time, format="%Y-%m") 

        df["exports"] = df["ALL_VAL_MO"].astype(float)
        
        df["description"] = df["E_COMMODITY_SDESC"]
            
        df["hs6"] = ecom
        # Now there whould only be one hs code per call, but I'm going use the groupby to be safe

        df.drop(["ALL_VAL_MO", "E_COMMODITY", "E_COMMODITY_SDESC", "COMM_LVL"], axis = 1, inplace = True)
        
        grp = df.groupby(["hs6"]) # group all exports over all the months (again there should only be one hs6 code)

        top_products = grp.agg({"exports":"sum", "description":"first"})
               
        top_products["hs-code"] = ecom
        
        top_products["tariff"] = tariff

        top_products["export_share"] = 100.*( top_products["exports"] / get_aggregate_hs6_exports(ecom) )

        top_products["color"] = "#ff0000"
    
        return top_products
    
    else:
        # some of the products have no trade, this just deals with these issues. 
    
        df = pd.DataFrame(columns=['exports', 'description', 'hs-code', 'tariff', 'export_share', 'color'])
        
        df.index.name = "hs6"
        
        new_row = {'exports': 0.0, 'description': " ", 'hs-code': ecom, 'tariff': tariff, 'export_share': 0.0, 'color': "#EE1C25"}
        # so if its empty, we just create a row with zero exports and the description comes from the
        # chinese side
        
        df = pd.concat([df, pd.DataFrame([new_row], index=[ecom])])
    
    return df

In [20]:
foo = get_exports_hs10("120190", 25.0)

In [21]:
foo

,exports,description,hs-code,tariff,export_share,color
hs6,,,,,,
120190,99082349.0,"SOYBEANS, NESOI",120190,25.0,0.404374,#ff0000


In [24]:
tariff_list.head()

,hs-code
0,0105.11.22
1,0105.94.92
2,0105.99.12
3,0207.11.91
4,0207.11.92


In [25]:
# this will then work through the tariff list and construct the data frame

trade_df_canada = pd.DataFrame(columns=['exports',"description",'hs-code'])

tariff_rate = 25.0

for index, row in canada_list.iterrows():
    
#     print(index)
    
#     print(row['hs-code'])
    
    foo = get_exports_hs10(row['hs6'], tariff_rate)
    
    trade_df_canada = pd.concat([trade_df_canada, foo])

In [26]:
trade_df_canada.sort_values(by = ["exports"], ascending = False)[0:20]

,exports,description,hs-code,tariff,export_share,color
210690,1.591995e+09,FOOD PREPARATIONS NESOI,210690,25.0,24.606500,#ff0000
190590,1.327189e+09,"BREAD, PASTRY, CAKES, ETC NESOI & PUDDINGS",190590,25.0,61.129086,#ff0000
330499,9.866899e+08,"BEAUTY & SKIN CARE PREPARATION, NESOI",330499,25.0,20.928865,#ff0000
401110,6.746851e+08,"NEW PNEUMATIC TIRES OF RUBBER, FOR MOTOR CARS",401110,25.0,46.397588,#ff0000
401120,6.447361e+08,"NEW PNEUMATIC TIRES OF RUBBER, FOR BUSES OR TR...",401120,25.0,35.776965,#ff0000
210390,6.253456e+08,SAUCES ETC. MIXED CONDIMENTS AND SEASONINGS NESOI,210390,25.0,35.986492,#ff0000
481910,5.434679e+08,"CARTONS, BOXES & CASES CORRUGATED PAPER & PAPERBD",481910,25.0,39.271127,#ff0000
711319,5.409617e+08,"JEWELRY AND PARTS THEREOF, OF OTH PRECIOUS METAL",711319,25.0,5.115475,#ff0000
220299,4.832143e+08,"NONALCOHOLIC BEVERAGES, NESOI",220299,25.0,54.040038,#ff0000
711021,4.699664e+08,"PALLADIUM, UNWROUGHT OR IN POWDER FORM",711021,25.0,19.662697,#ff0000


In [27]:
trade_df_canada.exports.sum()

39166104551.0

In [28]:
6.746851e+08 + 6.746851e+08

1349370200.0

In [ ]:
39,166,104,551


In [24]:
trade_df_10 = trade_df_10.astype({
    'hs-code': 'string',
})

pq.write_table(pa.Table.from_pandas(trade_df_10), "./data/china-10-percent-US-exports-030425.parquet")

trade_df_10.to_csv("./data/china-10-percent-US-exports-030425.csv", index=False)

In [91]:
trade_df_10.dtypes

exports        float64
description     object
hs-code         string
tariff         float64
dtype: object

In [25]:
# this will then work through the tariff list and construct the data frame

trade_df_15 = pd.DataFrame(columns=['exports',"description",'hs-code'])

tariff_rate = 15.0

for index, row in foo_list_15.iterrows():
    
#     print(index)
    
#     print(row['hs-code'])
    
    foo = get_exports_hs10(row['hs6'], tariff_rate)
    
    trade_df_15 = pd.concat([trade_df_15, foo])

In [26]:
trade_df_15.sort_values(by = ["exports"], ascending = False)[0:30]

,exports,description,hs-code,tariff,export_share,color
520100,1.487518e+09,"COTTON, NOT CARDED OR COMBED",520100,15.0,29.703900,#EE1C25
100199,5.569497e+08,"WHEAT AND MESLIN, NESOI",100199,15.0,9.683841,#EE1C25
020714,3.399914e+08,"CHICKEN CUTS AND EDIBLE OFFAL (INC LIVERS), FR...",020714,15.0,10.407037,#EE1C25
100590,3.309318e+08,"CORN (MAIZE), OTHER THAN SEED CORN",100590,15.0,2.354371,#EE1C25
160232,1.379402e+08,"PREPARED OR PRESERVED CHICKEN MEAT OR OFFAL, N...",160232,15.0,33.528331,#EE1C25
100191,2.950891e+06,"WHEAT AND MESLIN SEED, NESOI",100191,15.0,25.906791,#EE1C25
110100,5.734950e+05,WHEAT OR MESLIN FLOUR,110100,15.0,0.379538,#EE1C25
520300,3.420600e+05,"COTTON, CARDED OR COMBED",520300,15.0,4.022376,#EE1C25
020713,9.396200e+04,CHICKEN CUTS & EDIBLE OFFAL (INCL LIVER) FRSH/...,020713,15.0,0.009257,#EE1C25
020712,7.281400e+04,"MEAT & OFFAL OF CHICKENS,NOT CUT IN PIECES,FROZEN",020712,15.0,0.420715,#EE1C25


In [28]:
trade_df_15.exports.sum()

trade_df_15 = trade_df_15.astype({
    'hs-code': 'string',
})

pq.write_table(pa.Table.from_pandas(trade_df_15), "./data/china-15-percent-US-exports-030425.parquet")

trade_df_15.to_csv("./data/china-15-percent-US-exports-030425.csv", index=False)

In [27]:
trade_df_15.exports.sum()

2857368987.0

In [29]:
def make_source(df):
    
    df["position"] = df.reset_index().index.values
        
    df["hover_label"] = (df["exports"]/1000000000).map('{:,.1f}'.format)
    
    df["hover_label_2"] = (df["export_share"]).map('{:,.1f}'.format)
    
    df["hover_label_3"] = (df["tariff"]).map('{:,.1f}'.format)
    
    source = ColumnDataSource(df)
    
    return source

In [60]:
def make_bar_chart(df):

    width = 600
    height = 500

    source = make_source(df)
        
    p = figure(plot_height=height, plot_width = width, title= "Top US Exports to China Subject to Retaliatory Tariffs",
           toolbar_location = 'below',
           tools = "reset")
        
    p.vbar(x = "position", top = "exports", width = 0.6, alpha = 0.65,
       hatch_pattern = " ",hatch_alpha = 0.10, color = "color",
       source = source)
    
    y_custom = CustomJSHover(code=""" return '' + special_vars.data_y
            """)

##########################################################################
    TIMETOOLTIPS = """
    <div style="background-color:#F5F5F5; opacity: 0.95; border: 0px 0px 0px 0px">
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">@description</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">2024 Exports: $@hover_label Billion</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Share of Total: @hover_label_2%</span>
        </div>
        <div style = "text-align:left;">
            <span style="font-size: 13px; font-weight: bold">Applied Tariff: @hover_label_3%</span>
        </div>
    </div>
    """

    p.add_tools(HoverTool(tooltips = TIMETOOLTIPS))
##########################################################################

    #p.ygrid.grid_line_color = None
    p.xgrid.grid_line_color = None
    
    p.title.text_font_size = '13pt'
    p.xaxis.major_tick_line_color = None  # turn off x-axis major ticks
    p.xaxis.minor_tick_line_color = None  # turn off x-axis minor ticks
    p.xaxis.major_label_text_font_size = '0pt'  # turn off x-axis tick labels

    p.yaxis.formatter = NumeralTickFormatter(format="($0.0 a)")
    p.yaxis.minor_tick_line_color = None
    p.y_range.start = 0 
    
    p.y_range.end = df.exports.max() + 0.10*df.exports.max()
    
    p.border_fill_color = background    
    
    p.background_fill_color = background 
    p.background_fill_alpha = 0.75    
    
    p.toolbar.autohide = True
    
    p.outline_line_color = None
    p.sizing_mode= "scale_both"
    p.max_height = height
    p.max_width = width
    p.min_height = int(0.25*height)
    p.min_width = int(0.25*width)
    return p

In [61]:
p1 = make_bar_chart(trade_df_10.sort_values(by = ["exports"], ascending = False)[0:20])

p2 = make_bar_chart(trade_df_15.sort_values(by = ["exports"], ascending = False)[0:10])

In [62]:
tab1 = Panel(child= p1, title="10% Tariff")

tab2 = Panel(child= p2, title="15% Tariff")

output_file('.\\docs\\' + "china-retaliation.html")

div0 = Div(text = """Each bar represents the total sum of U.S. exports in 2024 for a HS6 category subject to Chinese tariffs. 
Only max 20 categories are displayed. Hover your cursor over each bar to learn more.
""", max_width=600, background = background )

div0.sizing_mode= "scale_both"
        
outfig = column(Tabs(tabs=[tab1, tab2], tabs_location = "above"), div0, sizing_mode="scale_both")

show(outfig)

In [30]:
crl = ["darkblue","slategray","slategray","crimson","crimson"]

background = "#ffffff"